In [54]:
import pandas as pds
import json 
import math as mt
df_raw_data = pds.read_csv(r'C:\Users\Chawanwit\Desktop\doc\bucket\ascentis\part-00000-3e003a3a-c205-4bdd-b203-7bb4eb36b325-c000.csv')
dfa_template = pds.read_csv(r'C:\Users\Chawanwit\Desktop\doc\bucket\ascentis\asc_json_mapper_template_19082022.csv')

In [55]:
dict_result_list = []
dict_list_temp = {}
dict_temp = {}
dict_temp_data = {}
dict_row_value = {}


dict_values_list = []
dict_value_data = {}


dict_row_group = {}
dict_fields = {} #--- feilds in space 1 
dict_json_row = {}
json_data_first_space = {}

In [56]:
#verify data group
def verify_group(dict_row):
    dict_row_group = dict_row["group"] 
    return dict_row_group
#get row feature
def get_row_feature(dict_row):
    dict_row_feature = dict_row["feature"]
    return dict_row_feature
#verify data type
def verify_data_type(dict_row):
    dict_row_data_type = dict_row["data_type"]
    #---- verify special data type
    return dict_row_data_type
#get row selection
def get_row_selection(dict_row):
    dict_row_selection = dict_row["selection"]
    return dict_row_selection
#get row name
def get_name(dict_row):
    dict_row_name = dict_row["name"]
    #---- verify special data type
    if pds.isna(dict_row_name):
        dict_row_name = ""
    return dict_row_name
#get row description
def get_row_description(dict_row):
    dict_row_description = dict_row["description"]
    if pds.isna(dict_row_description):
        dict_row_description = ""
    return dict_row_description
#get row aggregatable
def get_row_aggregatable(dict_row):
    dict_row_aggregatable = dict_row["aggregatable"]
    if pds.isna(dict_row_aggregatable):
        dict_row_aggregatable = ""
    return dict_row_aggregatable



In [57]:
#set json row [ { "feature" : set_fields_convert_data  }]
def set_row_to_json(dict_row):
    data_type  = verify_data_type(dict_row)
    dict_row_group = verify_group(dict_row)
    if pds.isna(dict_row_group): #------- select only group nan value
        dict_json_row[dict_row["feature"]] = set_fields_convert_data(dict_row)
    


In [58]:
#----- set info to categorical
def set_info_categorical_type(categorical_row):
    categorical_result = {}
    categorical_name = get_name(categorical_row)
    categorical_value = get_row_feature(categorical_row)
    categorical_result["name"] = categorical_name
    categorical_result["value"] = categorical_value
    return categorical_result

In [74]:
def get_categorical_value_from_source(categorical_name):
    categorical_result_list = []
    source_column_list = df_raw_data.columns.tolist()
    if categorical_name in source_column_list:
        #------ get direct unique value
        source_unique_values = sorted(df_raw_data[categorical_name].unique())
        if len(source_unique_values) > 0:
            for source_unique in source_unique_values:
                categorical_result = {}
                categorical_result["name"] = str(source_unique)
                categorical_result["value"] = str(source_unique)
                categorical_result_list.append(categorical_result)
    else:
        #------ find by like
        for source_column_name in source_column_list:
            if categorical_name in source_column_name:
                categorical_result = {}
                categorical_result["name"] = str(source_column_name)
                categorical_result["value"] = str(source_column_name)
                categorical_result_list.append(categorical_result)

    return categorical_result_list
    

In [60]:
#---- value will set when data type is categorical
def set_categorical_values(dict_row):
    data_categorical_row_group = verify_group(dict_row)
    data_categorical_row_type = verify_data_type(dict_row)
    data_categorical_row_feature = get_row_feature(dict_row)
    dict_categorical_row_values = []
    #----- verify categorical type---
    if data_categorical_row_type == "categorical":
        #-------------------------------- get category from dfa_template loc 
        categorical_row_list = dfa_template.loc[dfa_template["group"]==data_categorical_row_feature] 
        if len(categorical_row_list) > 0 :
            for categorical_row_index in range(len(categorical_row_list)):
                categorical_row = categorical_row_list.iloc[categorical_row_index]
                dict_categorical_row_values.append(set_info_categorical_type(categorical_row))      
        else :
            #------ when data type categorical but empty value
            dict_categorical_row_values = get_categorical_value_from_source(data_categorical_row_feature)
    else :
        dict_categorical_row_values = []
    return dict_categorical_row_values

In [61]:
#---- fields will set when data type is group
def set_group_fields(dict_row):
    data_group_row_feature = get_row_feature(dict_row)
    data_group_row_group = verify_group(dict_row)
    data_group_row_type = verify_data_type(dict_row)
    dict_group_fields_values = {}
    if data_group_row_type == "group":
        #------------- get loc from dfa_template
        group_row_list = dfa_template.loc[dfa_template["group"]==data_group_row_feature]
        if len(group_row_list) > 0 :
            #dict_group_fields = {}
            for group_row_index in range(len(group_row_list)):
                group_result_index = {}
                #--------------------------
                group_row = group_row_list.iloc[group_row_index]
                #--------------------------
                group_name = get_name(group_row)
                group_value = get_row_feature(group_row)
                group_value_data_type = verify_data_type(group_row)
                #--------------------------
                group_result_index["name"] = group_name
                group_result_index["data_type"] = group_value_data_type
                #-------------------------- 
                dict_group_fields_values[group_value] = group_result_index
        else : 
            #------ when data type group but empty value
            dict_group_fields_values = {}
                
                
        
    else :
        dict_group_fields_values = {}
    #----- find by name like
    return dict_group_fields_values

In [62]:
def set_float_min(source_float_column_name):
    dict_float_min_value = 0
    source_column_list = df_raw_data.columns.tolist()
    if source_float_column_name in source_column_list:
        #------ get direct unique value
        source_min_values = df_raw_data[source_float_column_name].min()
        dict_float_min_value = float(source_min_values)
    else:
        dict_float_min_value = float(0)
    
    return dict_float_min_value

In [63]:
def set_float_max(source_float_column_name):
    dict_float_max_value = 0
    source_column_list = df_raw_data.columns.tolist()
    if source_float_column_name in source_column_list:
        #------ get direct unique value
        source_max_values = df_raw_data[source_float_column_name].max()
        dict_float_max_value = float(source_max_values)
    else:
        dict_float_max_value = float(1)
    return dict_float_max_value
    

In [70]:
#----- detect data type and add fields
def set_fields_by_data_type(dict_row,dict_row_value):
    data_type = verify_data_type(dict_row)
    row_group = verify_group(dict_row)
    data_feature = get_row_feature(dict_row)
    dict_row_value_verify_type = {}
    if  data_type == "string":
        dict_row_value["aggregatable"] = get_row_aggregatable(dict_row)
    elif data_type == "categorical":
        dict_row_value["selection"] = get_row_selection(dict_row)
        dict_row_value["values"] = set_categorical_values(dict_row)
    elif data_type == "group":
        dict_row_value["fields"]= set_group_fields(dict_row)
    elif data_type == "float":
        dict_row_value["max_value"] = set_float_max(data_feature) #------------ get max from other source value
        dict_row_value["min_value"] = set_float_min(data_feature) #------------ get min from other source value
    
    return dict_row_value


In [71]:
def set_fields_convert_data(dict_row):
    dict_row_value = {}
    dict_row_value = {
    #---------- Default value ---------------
    "name":get_name(dict_row),
    "description":get_row_description(dict_row),
    "data_type":verify_data_type(dict_row),
    }
    dict_row_value.update(set_fields_by_data_type(dict_row,dict_row_value))
    return dict_row_value

In [72]:
def set_row_data(row_index):
    dict_row = dfa_template.iloc[row_index] #---- get iloc row
    #____________________________________________________
    dict_row_group = verify_group(dict_row) #---- group
    dict_row_feature = get_row_feature(dict_row) #---- feature
    dict_row_data_type = verify_data_type(dict_row) #---- data_type
    dict_row_selection = get_row_selection(dict_row) #---- selection
    dict_row_name = get_name(dict_row) #---- name 
    dict_row_description = get_row_description(dict_row) #---- description
    dict_row_aggregatable = get_row_description(dict_row) #---- aggregatable

    set_row_to_json(dict_row)
    return dict_row

In [75]:
for data_row in range(len(dfa_template)):
        set_row_data(data_row)

In [76]:
#----------- define function read csv template
def read_csv_template():
    for data_row in range(len(dfa_template)):
        set_row_data(data_row)
    #----- Json 1 space tab [model,client,fields]
    json_data_first_space = {
        "model": "@model_type",
        "client": "@client_customercode",
        "fields": dict_json_row
    }
    
    
    return json_data_first_space
    

In [77]:
read_csv_template()

{'model': '@model_type',
 'client': '@client_customercode',
 'fields': {'blastrecipient_member_id': {'name': 'Member ID',
   'description': '',
   'data_type': 'string',
   'aggregatable': True},
  'output_email': {'name': 'Respond to email',
   'description': 'Is this member likely to respond to the email campaign?',
   'data_type': 'boolean'},
  'output_sms': {'name': 'Respond to sms',
   'description': 'Is this member likely to respond to the SMS campaign?',
   'data_type': 'boolean'},
  'is_blast_on_weekday': {'name': 'Blast on weekday?',
   'description': 'Is this predicted blast on a weekday?',
   'data_type': 'boolean'},
  'is_blast_on_weekend': {'name': 'Blast on weekends?',
   'description': 'Is this predicted blast on a weekend?',
   'data_type': 'boolean'},
  'blast_on_day_of_week': {'name': 'Day of week',
   'description': 'Day of week of the predicted blast',
   'data_type': 'integer'},
  'blast_on_month_of_year': {'name': 'Month of year',
   'description': 'Month of year 